## Confidence interval - Stochastic Volatility

In [19]:
USER_PATH <- "D:/Softwares/"
USER_PATH2 <- "D:/Softwares/GitHub/Discrete-SV-Models-Estimation/Code/" # Change only here
ILLUS_PATH <- paste0(USER_PATH2, "Illustration/")
DATA_INPUT_PATH <- paste0(USER_PATH,"GitHub/Systemic_Risk_Measures/Code/Python/Data_Extraction/")
FUNCTION_PATH <- paste0(USER_PATH, "GitHub/Systemic_Risk_Measures/Code/R/Functions/")

In [20]:
source(paste0(FUNCTION_PATH, "CoVaR_project2.R"))
source(paste0(FUNCTION_PATH, "CoVaR_Functions.R"))

In [21]:
library(ggplot2)
library(dplyr)
library(tidyquant)
library(stats)
library(tibble)
library(base)
library(tidyverse)
library(reshape2)
library(pbivnorm)
library(MASS)
library(stargazer)

#### I) QML Method

In [22]:
sv_simul_can <- function(theta, log_h0, T){
    epsilon <- c()
    log_h <- c()
    omega <- theta[1]
    beta <- theta[2]
    sigma <- theta[3]
    eta <- rnorm(T, 0, 1)
    nu <- rnorm(T, 0, 1)
    
    log_h[1] <- omega + beta * log_h0 + sigma * nu[1]
    epsilon[1] <- sqrt(exp(log_h[1])) * eta[1]
    
    for (t in 2:T){
        log_h[t] <- omega + beta * log_h[t-1] + sigma * nu[t]
        epsilon[t] <- sqrt(exp(log_h[t])) * eta[t]
    }
    list(epsilon, log_h)
}

In [23]:
obj_sv <- function(theta, epsilon){
    alpha_t <- c()
    P_t <- c()
    F_t <- c()
    K_t <- c()
    omega <- theta[1]
    beta <- theta[2]
    sigma <- theta[3]
    mu_Z <- -1.270
    sigma_Z <- sqrt(pi**2/2)
    
    a0 <- 0
    beta0 <- 0.8
    P0 <- sigma**2
    F0 <- P0 + sigma_Z**2
    
    T <- length(epsilon)
    y_t <- log(epsilon**2)
    
    alpha_t[1] <- beta0 * a0 + omega
    P_t[1] <- beta**2 * P0 + sigma**2
    K_t[1] <- beta * P0 * 1/F0
    
    for (t in 2:T){
        F_t[t-1] <- P_t[t-1] + sigma_Z**2
        K_t[t] <- beta * P_t[t-1] * 1/F_t[t-1]
        alpha_t[t] <- beta * alpha_t[t-1] + K_t[t] *
            (y_t[t-1] - alpha_t[t-1] - mu_Z) + omega
        P_t[t] <- beta**2 * P_t[t-1] - K_t[t]**2 * F_t[t-1] + sigma**2
    }
    F_t[T] <- P_t[T-1] + sigma_Z**2
    qml <- -T/2 * log(2*pi) - 1/2*sum(log(F_t) +
        (log(epsilon**2) - alpha_t - mu_Z)**2 / F_t)
    -qml
}

In [24]:
estim_sv <- function(theta0, epsilon){
    valinit <- theta0
    lower <- c(-Inf, 0, 0)
    upper <- c(Inf, 0.999, Inf)
    res <- nlminb(valinit, obj_sv, lower=lower,
                  upper=upper, epsilon=epsilon)
    theta_hat <- res$par[1:3]
    theta_hat
}

In [25]:
# Inputs
omega <- -0.736
beta <- 0.9
sigma <- 0.363
theta <- c(omega, beta, sigma)

# Estimation Inputs
omega0 <- -0.6
beta0 <- 0.9
sigma0 <- 0.5
theta0 <- c(omega0, beta0, sigma0)

theta <- c(omega, beta, sigma)
log_h0 <- omega
T <- 3000
n <- 250
N <- T + n
M <- 1
S <- 100

alpha <- 0.05
alpha_l <- 0.025
alpha_h <- 0.0975

In [26]:
# Check algorithm running time
start.time <- Sys.time()

In [ ]:
# Initialistation
sigma2_hat_conso <- matrix(0, nrow=M, ncol=N)
eta_hat_conso <- matrix(0, nrow=M, ncol=N)
epsilon_conso <- matrix(0, nrow=M, ncol=N)
VaR_mat <- matrix(0, nrow=M, ncol=N)
xi_hat <- c()
eta_boot <- c()
xi_boot <- c()
lower_bound <- matrix(0, nrow=M, ncol=N)
upper_bound <- matrix(0, nrow=M, ncol=N)

set.seed(123)

for (j in 1:M){
    epsilon <- c()
    log_h <- c()
    omega <- theta[1]
    beta <- theta[2]
    sigma <- theta[3]
    eta <- rnorm(N, 0, 1)
    nu <- rnorm(N, 0, 1)
    
    log_h[1] <- omega + beta * log_h0 + sigma * nu[1]
    epsilon[1] <- sqrt(exp(log_h[1])) * eta[1]
    
    for (t in 2:N){
        log_h[t] <- omega + beta * log_h[t-1] + sigma * nu[t]
        epsilon[t] <- sqrt(exp(log_h[t])) * eta[t]
    }
    
    epsilon <- (epsilon+0.00001) * 100
    epsilon_conso[j,] <- epsilon
    
    ## 1) Stochastic Volatility Model
    estimation <- estim_sv(theta0, epsilon)
    theta_hat <- estimation
    theta_hat
    
    alpha_t <- c()
    P_t <- c()
    F_t <- c()
    K_t <- c()
    omega <- theta_hat[1]
    beta <- theta_hat[2]
    sigma <- theta_hat[3]
    mu_Z <- -1.270
    sigma_Z <- sqrt(pi**2/2)
    
    a0 <- 0
    beta0 <- 0.8
    P0 <- sigma**2
    F0 <- P0 + sigma_Z**2
    
    y_t <- log(epsilon**2)
    
    alpha_t[1] <- beta0 * a0 + omega
    P_t[1] <- beta**2 * P0 + sigma**2
    K_t[1] <- beta * P0 * 1/F0
    
    for (i in 2:N){
        F_t[i-1] <- P_t[i-1] + sigma_Z**2
        K_t[i] <- beta * P_t[i-1] * 1/F_t[i-1]
        alpha_t[i] <- beta * alpha_t[i-1] + K_t[i] *
                (y_t[i-1] - alpha_t[i-1] - mu_Z) + omega
        P_t[i] <- beta**2 * P_t[i-1] - K_t[i]**2 * F_t[i-1] + sigma**2
    }
    
    sigma2_hat_conso[j,] <- exp(alpha_t)
    eta_hat_conso[j,] <- epsilon / sqrt(exp(alpha_t))
    xi_hat[j] <- quantile(eta_hat_conso[j,1:T], probs = seq(0, 1, 0.01))[alpha*100+1]
    VaR_mat[j,] <- xi_hat[j] * sqrt(sigma2_hat_conso[j,])
    
    eta_hat <- eta_hat_conso[j,]
    
    # Confidence Interval - Bootstrap
    for (t in 1:N){
        for (s in 1:S){
            index <- floor(runif(T, min=1, max=T+1))
            for (k in 1:T){
                 eta_boot[k] <- (eta_hat[index[k]] - mean(eta_hat[index])) / sd(eta_hat[index])
            }
            xi_boot[s] <- quantile(eta_boot, probs = seq(0, 1, 0.001))[alpha*1000+1]
        }
        eps_boot <- sqrt(sigma2_hat_conso[j,t]) * xi_boot
        lower_bound[j,t] <- quantile(eps_boot, probs = seq(0, 1, 0.001))[alpha_l*1000+1]
        upper_bound[j,t] <- quantile(eps_boot, probs = seq(0, 1, 0.001))[alpha_h*1000+1]
    }
}

In [ ]:
end.time <- Sys.time()
time.taken<-round(end.time-start.time,2)
time.taken

In [ ]:
z <- 1
selection <- (T+1):N
plot(epsilon_conso[z,selection], type='l', col='black')
lines(VaR_mat[z,selection], type='l', col='blue')
lines(lower_bound[z,selection], type='l', col='cyan')
lines(upper_bound[z,selection], type='l', col='cyan')

In [ ]:
mean(eta_boot)
sd(eta_boot)

In [ ]:
# stocks <- stocks[,5]
stocks <- system

In [ ]:
selection <- 2500:3000
epsilon <- stocks[stocks!=0] * 100
epsilon <- epsilon[selection]
DATE <- as.Date(returns[selection,1])
DATE_VIX <- as.Date(date_vix)
T <- length(epsilon)

In [ ]:
plot(epsilon, type='l')

In [ ]:
plot(vix_dat, type='l')


In [ ]:
# Estimation Inputs
omega0 <- -0.6
beta0 <- 0.9
sigma0 <- 0.5
theta0 <- c(omega0, beta0, sigma0)

### A/ Stochastic Volatility Model

In [ ]:
estimation <- estim_sv(theta0, epsilon)
theta_hat <- estimation
theta_hat

In [ ]:
alpha_t <- c()
P_t <- c()
F_t <- c()
K_t <- c()
omega <- theta_hat[1]
beta <- theta_hat[2]
sigma <- theta_hat[3]
mu_Z <- -1.270
sigma_Z <- sqrt(pi**2/2)
    
a0 <- 0
beta0 <- 0.8
P0 <- sigma**2
F0 <- P0 + sigma_Z**2
    
T <- length(epsilon)
y_t <- log(epsilon**2)
    
alpha_t[1] <- beta0 * a0 + omega
P_t[1] <- beta**2 * P0 + sigma**2
K_t[1] <- beta * P0 * 1/F0
    
for (t in 2:T){
    F_t[t-1] <- P_t[t-1] + sigma_Z**2
    K_t[t] <- beta * P_t[t-1] * 1/F_t[t-1]
    alpha_t[t] <- beta * alpha_t[t-1] + K_t[t] *
            (y_t[t-1] - alpha_t[t-1] - mu_Z) + omega
    P_t[t] <- beta**2 * P_t[t-1] - K_t[t]**2 * F_t[t-1] + sigma**2
}

## GARCH Model

In [ ]:
omega_0 <- -0.6
alpha_0 <- 0.1
beta_0 <- 0.85

In [ ]:
# Initialisation
sigma2_hat <- c()
eta_hat <- c()

estimation <- estimgarch11(omega_0, alpha_0, beta_0, epsilon)
theta_hat_g <- estimation$coef

omega_hat_g <- theta_hat_g[1]
alpha_hat_g <- theta_hat_g[2]
beta_hat_g <- theta_hat_g[3]

sigma2_hat[1]<-omega_hat_g
eta_hat[1] <- epsilon[1] / sqrt(sigma2_hat[1])
for (i in 2:T){
    sigma2_hat[i]<-omega_hat_g + alpha_hat_g * epsilon[i-1]**2 + beta_hat_g * sigma2_hat[i-1]
    eta_hat[i] <- epsilon[i] / sqrt(sigma2_hat[i])
}

In [ ]:
# Load Data
t<-1:T
col_names<-c('t', 'returns', 'stochastic volatility', 'garch volatility')
confidence_tab <- as.data.frame(epsilon)
confidence_tab <- cbind(confidence_tab, sqrt(exp(alpha_t)))
confidence_tab <- cbind(confidence_tab, sqrt(sigma2_hat))
confidence_tab <- cbind(t, confidence_tab)
colnames(confidence_tab) <- col_names
confidence_tab <- melt(confidence_tab, id="t")
confidence_tab <- cbind(rep(DATE[1:T]), confidence_tab)
colnames(confidence_tab) <- c('Date', 't', 'variable', 'value')

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 100)
confidence_chart <-ggplot(data=confidence_tab, aes(x=Date, y=value, colour=variable))+
    geom_line(lwd=0.5)+
    theme_classic()+
    scale_color_manual(name = "Variables", values= c('black', 'blue', 'red'),
                       labels = c('Returns', 'Stochastic Volatility', 'Garch Volatility'))+
    labs(title = "S&P 500 Returns vs. Stochastic & GARCH Volatility"
         , x = "Time", y="Value")+
    theme(
        axis.title = element_text(size =12),
        axis.text = element_text(size = 12, color = 'black')
    )

In [ ]:
confidence_chart

In [ ]:
# pdf(file=paste0(ILLUS_PATH, "QML_volat_RD.pdf"),width=10, height=5)
# confidence_chart
# dev.off()

In [ ]:
vix_100 <- vix_dat[1:501] / mean(vix_dat[1:501]) * 100
garch_100 <- sqrt(sigma2_hat) / mean(sqrt(sigma2_hat)) * 100

In [ ]:
length(vix_100)

In [ ]:
# Load Data
t<-1:T
col_names<-c('t', 'vix', 'garch volatility')
confidence_tab_vix <- as.data.frame(vix_100)
confidence_tab_vix <- cbind(confidence_tab_vix, garch_100)
confidence_tab_vix <- cbind(t, confidence_tab_vix)
colnames(confidence_tab_vix) <- col_names
confidence_tab_vix <- melt(confidence_tab_vix, id="t")
confidence_tab_vix <- cbind(rep(DATE[1:T]), confidence_tab_vix)
colnames(confidence_tab_vix) <- c('Date', 't', 'variable', 'value')

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 100)
confidence_chart_vix <-ggplot(data=confidence_tab_vix, aes(x=Date, y=value, colour=variable))+
    geom_line(lwd=0.5)+
    theme_classic()+
    scale_color_manual(name = "Variables", values= c('blue', 'red'),
                       labels = c('Vix', 'GARCH Volatility'))+
    labs(title = "S&P 500 Returns vs. VIX"
         , x = "Time", y="Value")+
    theme(
        axis.title = element_text(size =12),
        axis.text = element_text(size = 12, color = 'black')
    )

In [ ]:
confidence_chart_vix

#### Spread

In [ ]:
# Load Data
t<-1:T
col_names<-c('t', 'volatility risk premium', 'zero line')
confidence_tab_spread <- as.data.frame(vix_100 - garch_100)
confidence_tab_spread <- cbind(confidence_tab_spread, rep(0, 501))
confidence_tab_spread <- cbind(t, confidence_tab_spread)
colnames(confidence_tab_spread) <- col_names
confidence_tab_spread <- melt(confidence_tab_spread, id="t")
confidence_tab_spread <- cbind(rep(DATE[1:T]), confidence_tab_spread)
colnames(confidence_tab_spread) <- c('Date', 't', 'variable', 'value')

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 100)
confidence_chart_spread <-ggplot(data=confidence_tab_spread, aes(x=Date, y=value, colour=variable))+
    geom_line(lwd=0.5)+
    theme_classic()+
    scale_color_manual(name = "Variables", values= c('black', 'red'),
                       labels = c('Volatility Risk Premium', 'Zero Line'))+
    labs(title = "Volatility Risk Premium"
         , x = "Time", y="Value")+
    theme(
        axis.title = element_text(size =12),
        axis.text = element_text(size = 12, color = 'black')
    )

In [ ]:
confidence_chart_spread

#### Risk

#### A/ Stochastic Volatility

In [ ]:
alpha1 <- 0.20
eta_hat <- epsilon / sqrt(exp(alpha_t))
xi_hat <- quantile(eta_hat, probs = seq(0, 1, 0.01))[alpha1*100+1]
VaR_SV <- sqrt(exp(alpha_t)) * xi_hat

#### B/ GARCH

In [ ]:
eta_hat_g <- epsilon / sqrt(sigma2_hat)
xi_hat_g <- quantile(eta_hat_g, probs = seq(0, 1, 0.01))[alpha1*100+1]
VaR_G <- sqrt(sigma2_hat) * xi_hat_g

#### C/ UNCONDITIONAL

In [ ]:
VaR_U <- quantile(epsilon, probs = seq(0, 1, 0.01))[alpha1*100+1]

In [ ]:
col_names<-c('t', 'returns', 'VaR_SV', 'VaR_G', 'VaR_U')
confidence_tab_var <- as.data.frame(epsilon)
confidence_tab_var <- cbind(confidence_tab_var, VaR_SV)
confidence_tab_var <- cbind(confidence_tab_var, VaR_G)
confidence_tab_var <- cbind(confidence_tab_var, VaR_U)
confidence_tab_var <- cbind(t, confidence_tab_var)
colnames(confidence_tab_var) <- col_names
confidence_tab_var <- melt(confidence_tab_var, id="t")
confidence_tab_var <- cbind(rep(DATE[1:T]), confidence_tab_var)
colnames(confidence_tab_var) <- c('Date', 't', 'variable', 'value')

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 100)
confidence_chart_var <-ggplot(data=confidence_tab_var, aes(x=Date, y=value, colour=variable))+
    geom_line(lwd=0.5)+
    theme_classic()+
    scale_color_manual(name = "Variables", values= c('black', 'red', 'blue', 'cyan'),
                       labels = c('Returns', '-VaR SV', '-VaR Garch', '-Unconditional VaR'))+
    labs(title = "Returns vs. VaR"
         , x = "Time", y="Value")+
    theme(
        axis.title = element_text(size =12),
        axis.text = element_text(size = 12, color = 'black')
    )

In [ ]:
confidence_chart_var

In [ ]:
pdf(file=paste0(ILLUS_PATH, "VaR_RD_020.pdf"),width=10, height=5)
confidence_chart_var
dev.off()

### Expected Shortfall

In [ ]:
ES_g <- c()
id_g <- c()
ES_sv <- c()
id_sv <- c()

ES_u <- rep(quantile(epsilon, probs = seq(0, 1, 0.01))[alpha1*100+1], 500)

for (t in 1:T){
    if (eta_hat_g[t] < xi_hat_g){
        id_g[t] <- 1
    } else {
        id_g[t] <- 0
    }
}   
eta_hat_g_mod <- id_g * eta_hat_g
eta_hat_g_mod <- eta_hat_g_mod[eta_hat_g_mod!=0]

ES_g <- mean(eta_hat_g_mod) * sqrt(exp(alpha_t))

for (t in 1:T){
    if (eta_hat[t] < xi_hat){
        id_sv[t] <- 1
    } else {
        id_sv[t] <- 0
    }
}   

eta_hat_sv_mod <- id_sv * eta_hat
eta_hat_sv_mod <- eta_hat_sv_mod[eta_hat_sv_mod!=0]

ES_sv <- mean(eta_hat_sv_mod) * sqrt(sigma2_hat)

In [ ]:
plot(epsilon, type='l', col='black')
lines(ES_sv, type='l', col='blue')
lines(ES_g, type='l', col='red')
lines(ES_u, type='l', col='cyan')

In [ ]:
# Load Data
t<-1:T
col_names<-c('t', 'Log-Returns', 'ES Stochastic Volatility', 'ES GARCH', 'ES Unconditional')
es_tab <- as.data.frame(epsilon[1:T])
es_tab <- cbind(es_tab, ES_sv[1:T])
es_tab <- cbind(es_tab, ES_g[1:T])
es_tab <- cbind(es_tab, ES_u[1:T])
es_tab <- cbind(t, es_tab)
colnames(es_tab) <- col_names
es_tab <- melt(es_tab, id="t")
es_tab <- cbind(rep(DATE[1:T]), es_tab)
colnames(es_tab) <- c('Date', 't', 'variable', 'value')

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 100)
es_chart <-ggplot(data=es_tab, aes(x=Date, y=value, colour=variable))+
    geom_line(lwd=0.5)+
    theme_classic()+
    scale_color_manual(name = "Variables", values= c('black', 'red', 'blue', 'cyan'),
                       labels = c('Returns', 'Expected Shortfall SV',
                                  'Expected  Shortfall Garch',
                                  'Expected Shortfall VaR'))+
    labs(title = "Returns vs. Expected Shortfall"
         , x = "Time", y="Value")+
    theme(
        axis.title = element_text(size =12),
        axis.text = element_text(size = 12, color = 'black')
    )

In [ ]:
es_chart

In [ ]:
pdf(file=paste0(ILLUS_PATH, "ES_RD_020.pdf"),width=10, height=5)
es_chart
dev.off()

### Monte Carlo - on Real Data

In [ ]:
T <- 3000
n <- 250
N <- T+n
M <- length(returns[1,]) - 1
epsilon_conso <- returns[,2:(M+1)]

In [ ]:
# Initialistation
sigma2_hat_g <- matrix(0, nrow=M, ncol=N)
eta_hat_g <- matrix(0, nrow=M, ncol=N)
VaR_mat_g <- matrix(0, nrow=M, ncol=N)
xi_hat_g <- c()
    
sigma2_hat_sv <- matrix(0, nrow=M, ncol=N)
eta_hat_sv <- matrix(0, nrow=M, ncol=N)
VaR_mat_sv <- matrix(0, nrow=M, ncol=N)
xi_hat_sv <- c()

VaR_mat_u <- matrix(0, nrow=M, ncol=N)

for (j in 1:M){
    epsilon <- epsilon_conso[,j]
    epsilon <- (epsilon+0.00001) * 100
    
    ## 1) Stochastic Volatility Model
    estimation <- estim_sv(theta0, epsilon)
    theta_hat <- estimation
    theta_hat
    
    alpha_t <- c()
    P_t <- c()
    F_t <- c()
    K_t <- c()
    omega <- theta_hat[1]
    beta <- theta_hat[2]
    sigma <- theta_hat[3]
    mu_Z <- -1.270
    sigma_Z <- sqrt(pi**2/2)
    
    a0 <- 0
    beta0 <- 0.8
    P0 <- sigma**2
    F0 <- P0 + sigma_Z**2
    
    y_t <- log(epsilon**2)
    
    alpha_t[1] <- beta0 * a0 + omega
    P_t[1] <- beta**2 * P0 + sigma**2
    K_t[1] <- beta * P0 * 1/F0
    
    for (i in 2:N){
        F_t[i-1] <- P_t[i-1] + sigma_Z**2
        K_t[i] <- beta * P_t[i-1] * 1/F_t[i-1]
        alpha_t[i] <- beta * alpha_t[i-1] + K_t[i] *
                (y_t[i-1] - alpha_t[i-1] - mu_Z) + omega
        P_t[i] <- beta**2 * P_t[i-1] - K_t[i]**2 * F_t[i-1] + sigma**2
    }
    
    sigma2_hat_sv[j,] <- exp(alpha_t)
    eta_hat_sv[j,] <- epsilon / sqrt(exp(alpha_t))
    xi_hat_sv[j] <- quantile(eta_hat_sv[j,1:T], probs = seq(0, 1, 0.01))[alpha1*100+1]
    VaR_mat_sv[j,] <- xi_hat_sv[j] * sqrt(sigma2_hat_sv[j,])
    
    ## 2) GARCH
    estimation_g <- estimgarch11(omega_0, alpha_0, beta_0, epsilon)
    theta_hat_g <- estimation_g$coef

    omega_hat_g <- theta_hat_g[1]
    alpha_hat_g <- theta_hat_g[2]
    beta_hat_g <- theta_hat_g[3]

    sigma2_hat_g[j,1]<-omega_hat_g
    eta_hat_g[j,1] <- epsilon[1] / sqrt(sigma2_hat_g[j,1])
    for (i in 2:N){
        sigma2_hat_g[j,i]<-omega_hat_g + alpha_hat_g * epsilon[i-1]**2 +
            beta_hat_g * sigma2_hat_g[j,i-1]
        eta_hat_g[j,i] <- epsilon[i] / sqrt(sigma2_hat_g[j,i])
    }
    xi_hat_g[j] <- quantile(eta_hat_g[j,1:T], probs = seq(0, 1, 0.01))[alpha1*100+1]
    VaR_mat_g[j,] <- xi_hat_g[j] * sqrt(sigma2_hat_g[j,])
    
    ## 3) unconditional Model
    VaR_mat_u[j,] <- quantile(epsilon[1:T], probs = seq(0, 1, 0.01))[alpha1*100+1]
}

In [ ]:
z <- 4
epsilon <- epsilon_conso[,z]
epsilon <- (epsilon+0.00001) * 100
plot(epsilon, type='l', col='black')
lines(sqrt(sigma2_hat_g[z,]), type='l', col='blue')
lines(sqrt(sigma2_hat_sv[z,]), type='l', col='red')

In [ ]:
z <- 4
plot(epsilon, type='l', col='black')
lines(VaR_mat_g[z,], type='l', col='blue')
lines(VaR_mat_sv[z,], type='l', col='red')

## Functions

In [ ]:
Hit_conso_g <- matrix(0, nrow=M, ncol=n)
Hit_conso_sv <- matrix(0, nrow=M, ncol=n)
Hit_conso_u <- matrix(0, nrow=M, ncol=n)

for (j in 1:M){
    epsilon <- epsilon_conso[,j]
    epsilon <- (epsilon+0.00001) * 100
    VaR_g <- VaR_mat_g[j,]
    VaR_sv <- VaR_mat_sv[j,]
    VaR_u <- VaR_mat_u[j,]
    for (t in (T+1):N){
        if (epsilon[t] < VaR_g[t]){
            Hit_conso_g[j,t-T] <- 1
        }
        if (epsilon[t] < VaR_sv[t]){
            Hit_conso_sv[j,t-T] <- 1
        }
        if (epsilon[t] < VaR_u[t]){
            Hit_conso_u[j,t-T] <- 1
        }
    }
}

### Backtests

In [ ]:
chi_level <- c(3.841)

In [ ]:
# STOCHASTIC VOLATILITY MODEL
########################################
###### UNCONDITIONAL COVERAGE TEST #####
########################################

UC_test_SV <- LR_uc_test(Hit_conso_sv, alpha1, chi_level)

UC_test_SV$Test_uc # To be compared with 5%

In [ ]:
# GARCH MODEL
########################################
###### UNCONDITIONAL COVERAGE TEST #####
########################################

UC_test_G <- LR_uc_test(Hit_conso_g, alpha1, chi_level)

UC_test_G$Test_uc # To be compared with 5%

In [ ]:
# unconditional MODEL
########################################
###### UNCONDITIONAL COVERAGE TEST #####
########################################

UC_test_U <- LR_uc_test(Hit_conso_u, alpha1, chi_level)

UC_test_U$Test_uc # To be compared with 5%

In [ ]:
# STOCHASTIC VOLATILITY MODEL
########################################
########## INDEPENDENCE TEST ###########
########################################

Ind_test_SV <- LR_ind_test(Hit_conso_sv, alpha1, chi_level)

Ind_test_SV$Test_ind # To be compared with 5%

In [ ]:
# GARCH MODEL
########################################
########## INDEPENDENCE TEST ###########
########################################

Ind_test_G <- LR_ind_test(Hit_conso_g, alpha1, chi_level)

Ind_test_G$Test_ind # To be compared with 5%

In [ ]:
# UNCONDITIONAL MODEL
########################################
########## INDEPENDENCE TEST ###########
########################################

Ind_test_U <- LR_ind_test(Hit_conso_u, alpha1, chi_level)

Ind_test_U$Test_ind # To be compared with 5%

In [ ]:
chi_level2 <- 5.991

In [ ]:
CC_test_G <- LR_cc_test(UC_test_G$LR_uc, Ind_test_G$LR_ind, alpha1, chi_level2)$Test_cc
CC_test_SV <- LR_cc_test(UC_test_SV$LR_uc, Ind_test_SV$LR_ind, alpha1, chi_level2)$Test_cc
CC_test_U <- LR_cc_test(UC_test_U$LR_uc, Ind_test_U$LR_ind, alpha1, chi_level2)$Test_cc

In [ ]:
alpha_crit_g <- alpha_crit((epsilon_conso[(T+1):N,]+0.00001) * 100, VaR_mat_g[,(T+1):N], alpha1)
alpha_crit_sv <- alpha_crit((epsilon_conso[(T+1):N,]+0.00001) * 100, VaR_mat_sv[,(T+1):N], alpha1)
alpha_crit_u <- alpha_crit((epsilon_conso[(T+1):N,]+0.00001) * 100, VaR_mat_u[,(T+1):N], alpha1)

In [ ]:
results_SV <- c(UC_test_SV$Test_uc, Ind_test_SV$Test_ind, CC_test_SV, mean(alpha_crit_sv$crit))
results_GARCH <- c(UC_test_G$Test_uc, Ind_test_G$Test_ind, CC_test_G, mean(alpha_crit_g$crit))
results_U <- c(UC_test_U$Test_uc, Ind_test_U$Test_ind, CC_test_U, mean(alpha_crit_u$crit))

#### Results

In [ ]:
Test_table <- matrix(0, nrow=3, ncol=4)
row_name <- c('SV Model', 'GARCH Model', 'Unconditional Model')
col_name <- c('UC', 'Ind.', 'CC', 'alpha-criterion')
Test_table[1,] <- results_SV
Test_table[2,] <- results_GARCH
Test_table[3,] <- results_U
Test_table <- as.data.frame(Test_table)
rownames(Test_table) <- row_name
colnames(Test_table) <- col_name

In [ ]:
Test_table <- round_df(Test_table, 3)
Test_table

In [ ]:
stargazer(Test_table,  summary=FALSE, covariate.labels=c("A","b","c"),
          title = "Backtesting Results - alpha=0.01",
out = paste0(ILLUS_PATH, "BTEST_Results_SV_001.tex"))